# MNIST BDP

The following is a breakdown of the problem of using a supervised learning model on the MNIST dataset. We begin with an empty project and fill it out with the pybdp interface to create an example of a machine learning block diagram for loading MNIST.

In [1]:
import json
import pybdp
from IPython.display import Markdown
from pprint import pprint

# Start with an empty project
project = pybdp.create_empty_project()

## System High Level

At the highest level, there is a processor that takes a model in and returns evaluation metrics. This is our "MNIST Experiment" processor.

We first create the spaces, block and processor then we display it with create_mermaid_graphic and also check out its ports and terminals.

In [2]:
# Add the spaces
project.add_space(id = "Model",
                  name = "Model",
                  description = "A machine learning model")
project.add_space(id = "Evaluation Metrics",
                  name = "Evaluation Metrics",
                  description = "Metrics for evaluating the model")

# Add the block
project.add_block(id="Experiment",
                  name="Experiment",
                  description="A machine learning experiment",
                  domain=["Model"],
                  codomain=["Evaluation Metrics"],)

# Add the processor
project.add_processor(id="MNIST Experiment",
                      name="MNIST Experiment",
                      description="Runs experiments on the MNIST dataset",
                      parent_id="Experiment",)


processor = project.processors_map["MNIST Experiment"]
display(Markdown(processor.create_mermaid_graphic()[0]))
print("Ports:")
print(processor.ports)
print("Terminals:")
print(processor.terminals)

```mermaid
---
config:
    layout: elk
---
graph LR
subgraph G0[MNIST Experiment - Experiment Block]
direction LR
X0[MNIST Experiment]
subgraph G0P[Ports]
direction TB
XX0P0[Model]
end
XX0P0[Model] o--o X0
subgraph G0T[Terminals]
direction TB
XX0T0[Evaluation Metrics]
end
X0 o--o XX0T0[Evaluation Metrics]
end

```

Ports:
[< Space ID: Model Name: Model >]
Terminals:
[< Space ID: Evaluation Metrics Name: Evaluation Metrics >]


In [ ]:
{
                    "Description": "Runs experiments on the MNIST dataset",
                    "ID": "MNIST Experiment",
                    "Name": "MNIST Experiment",
                    "Parent": "Experiment",
                    "Ports": [
                      "Model"
                    ],
                    "Subsystem": {
                      "Port Mappings": [
                        {
                          "Index": 0,
                          "Processor": "Default Supervised Learning"
                        }
                      ],
                      "System ID": "MNIST Experiment System",
                      "Terminal Mappings": [
                        {
                          "Index": 0,
                          "Processor": "Default Supervised Learning"
                        }
                      ]
                    },
                    "Terminals": [
                      "Evaluation Metrics"
                    ]
                  },

In [21]:
processor = project.processors_map["MNIST Experiment"]
display(Markdown(processor.create_mermaid_graphic()[0]))
print("Ports:")
print(processor.ports)
print("Terminals:")
print(processor.terminals)

```mermaid
---
config:
    layout: elk
---
graph LR
subgraph G0[MNIST Experiment - Experiment Block]
direction LR
X0[MNIST Experiment]
subgraph G0P[Ports]
direction TB
XX0P0[Model]
end
XX0P0[Model] o--o X0
subgraph G0T[Terminals]
direction TB
XX0T0[Evaluation Metrics]
end
X0 o--o XX0T0[Evaluation Metrics]
end

```

Ports:
[< Space ID: Model Name: Model >]
Terminals:
[< Space ID: Evaluation Metrics Name: Evaluation Metrics >]


## Breaking it down into data and model fitting/evaluation

This processor has a subsystem however which is evidenced by the fact that the processor is not primitive.

In [22]:
print(processor.is_primitive())

False


The create_mermaid_graphic_composite function lets us peek into the subsystem it represents.

In [23]:
display(Markdown(processor.create_mermaid_graphic_composite()[0]))

```mermaid
---
config:
    layout: elk
---
graph LR
subgraph GC0[MNIST Experiment - Experiment Block]
direction LR
subgraph GS0[MNIST Experiment System]
subgraph G1[Load MNIST - Load Supervised Features Block]
direction LR
X1[Load MNIST]
subgraph G1P[Ports]
direction TB
end
subgraph G1T[Terminals]
direction TB
XX1T0[X Train Array]
XX1T1[Y Train Array]
XX1T2[X Test Array]
XX1T3[Y Test Array]
end
X1 o--o XX1T0[X Train Array]
X1 o--o XX1T1[Y Train Array]
X1 o--o XX1T2[X Test Array]
X1 o--o XX1T3[Y Test Array]
end
subgraph G2[Default Supervised Learning - Supervised Learning Block]
direction LR
X2[Default Supervised Learning]
subgraph G2P[Ports]
direction TB
XX2P0[Model]
XX2P1[X Train Array]
XX2P2[Y Train Array]
XX2P3[X Test Array]
XX2P4[Y Test Array]
end
XX2P0[Model] o--o X2
XX2P1[X Train Array] o--o X2
XX2P2[Y Train Array] o--o X2
XX2P3[X Test Array] o--o X2
XX2P4[Y Test Array] o--o X2
subgraph G2T[Terminals]
direction TB
XX2T0[Evaluation Metrics]
end
X2 o--o XX2T0[Evaluation Metrics]
end
XX1T0[X Train Array] ---> XX2P1[X Train Array]
XX1T1[Y Train Array] ---> XX2P2[Y Train Array]
XX1T2[X Test Array] ---> XX2P3[X Test Array]
XX1T3[Y Test Array] ---> XX2P4[Y Test Array]
end
subgraph GC0P[Ports]
direction TB
X1P0[Model]
end
X1P0[Model] o--o XX2P0[Model]
subgraph GC0T[Terminals]
direction TB
X1T0[Evaluation Metrics]
end
XX2T0[Evaluation Metrics] o--o X1T0[Evaluation Metrics]
end

```

By calling get_system and grabbing the processors we can find the two inner components.

In [24]:
components = processor.get_system().processors
pprint(components)

[< Processor ID: Load MNIST Name: Load MNIST []->['X Train Array', 'Y Train Array', 'X Test Array', 'Y Test Array']>,
 < Processor ID: Default Supervised Learning Name: Default Supervised Learning ['Model', 'X Train Array', 'Y Train Array', 'X Test Array', 'Y Test Array']->['Evaluation Metrics']>]


Display the components and check if they are primitive.

In [29]:
for component in components:
    print("Processor: {}".format(component.name))
    print("Is primitive? - {}".format(component.is_primitive()))
    print()
    
    display(Markdown(component.create_mermaid_graphic()[0]))

Processor: Load MNIST
Is primitive? - False



```mermaid
---
config:
    layout: elk
---
graph LR
subgraph G0[Load MNIST - Load Supervised Features Block]
direction LR
X0[Load MNIST]
subgraph G0P[Ports]
direction TB
end
subgraph G0T[Terminals]
direction TB
XX0T0[X Train Array]
XX0T1[Y Train Array]
XX0T2[X Test Array]
XX0T3[Y Test Array]
end
X0 o--o XX0T0[X Train Array]
X0 o--o XX0T1[Y Train Array]
X0 o--o XX0T2[X Test Array]
X0 o--o XX0T3[Y Test Array]
end

```

Processor: Default Supervised Learning
Is primitive? - False



```mermaid
---
config:
    layout: elk
---
graph LR
subgraph G0[Default Supervised Learning - Supervised Learning Block]
direction LR
X0[Default Supervised Learning]
subgraph G0P[Ports]
direction TB
XX0P0[Model]
XX0P1[X Train Array]
XX0P2[Y Train Array]
XX0P3[X Test Array]
XX0P4[Y Test Array]
end
XX0P0[Model] o--o X0
XX0P1[X Train Array] o--o X0
XX0P2[Y Train Array] o--o X0
XX0P3[X Test Array] o--o X0
XX0P4[Y Test Array] o--o X0
subgraph G0T[Terminals]
direction TB
XX0T0[Evaluation Metrics]
end
X0 o--o XX0T0[Evaluation Metrics]
end

```

## One Level Deeper

Since both are not primitive, we can also show their subystems. Note that there can be infinite nesting, but for this block diagram this is the final level of recursion.

In [32]:
for component in components:
    print("Processor: {}".format(component.name))
    print()
    
    display(Markdown(component.create_mermaid_graphic_composite()[0]))

Processor: Load MNIST



```mermaid
---
config:
    layout: elk
---
graph LR
subgraph GC0[Load MNIST - Load Supervised Features Block]
direction LR
subgraph GS0[Load MNIST System]
subgraph G1[Load MNIST Dataset - Load Supervised Dataset Block]
direction LR
X1[Load MNIST Dataset]
subgraph G1P[Ports]
direction TB
end
subgraph G1T[Terminals]
direction TB
XX1T0[X]
XX1T1[Y]
end
X1 o--o XX1T0[X]
X1 o--o XX1T1[Y]
end
subgraph G2[Test-Train Split - Cross Validation Split Block]
direction LR
X2[Test-Train Split]
subgraph G2P[Ports]
direction TB
XX2P0[X]
XX2P1[Y]
end
XX2P0[X] o--o X2
XX2P1[Y] o--o X2
subgraph G2T[Terminals]
direction TB
XX2T0[X Train Array]
XX2T1[Y Train Array]
XX2T2[X Test Array]
XX2T3[Y Test Array]
end
X2 o--o XX2T0[X Train Array]
X2 o--o XX2T1[Y Train Array]
X2 o--o XX2T2[X Test Array]
X2 o--o XX2T3[Y Test Array]
end
subgraph G3[Image Normalization Preprocessing - Training - Training Data Preprocessing Block]
direction LR
X3[Image Normalization Preprocessing - Training]
subgraph G3P[Ports]
direction TB
XX3P0[X Train Array]
XX3P1[Y Train Array]
end
XX3P0[X Train Array] o--o X3
XX3P1[Y Train Array] o--o X3
subgraph G3T[Terminals]
direction TB
XX3T0[X Train Array]
XX3T1[Y Train Array]
end
X3 o--o XX3T0[X Train Array]
X3 o--o XX3T1[Y Train Array]
end
subgraph G4[Image Normalization Preprocessing - Testing - Testing Data Preprocessing Block]
direction LR
X4[Image Normalization Preprocessing - Testing]
subgraph G4P[Ports]
direction TB
XX4P0[X Train Array]
XX4P1[Y Train Array]
XX4P2[X Test Array]
XX4P3[Y Test Array]
end
XX4P0[X Train Array] o--o X4
XX4P1[Y Train Array] o--o X4
XX4P2[X Test Array] o--o X4
XX4P3[Y Test Array] o--o X4
subgraph G4T[Terminals]
direction TB
XX4T0[X Test Array]
XX4T1[Y Test Array]
end
X4 o--o XX4T0[X Test Array]
X4 o--o XX4T1[Y Test Array]
end
XX1T0[X] ---> XX2P0[X]
XX1T1[Y] ---> XX2P1[Y]
XX2T0[X Train Array] ---> XX3P0[X Train Array]
XX2T1[Y Train Array] ---> XX3P1[Y Train Array]
XX2T0[X Train Array] ---> XX4P0[X Train Array]
XX2T1[Y Train Array] ---> XX4P1[Y Train Array]
XX2T2[X Test Array] ---> XX4P2[X Test Array]
XX2T3[Y Test Array] ---> XX4P3[Y Test Array]
end
subgraph GC0P[Ports]
direction TB
end
subgraph GC0T[Terminals]
direction TB
X1T0[X Train Array]
X1T1[Y Train Array]
X1T2[X Test Array]
X1T3[Y Test Array]
end
XX3T0[X Train Array] o--o X1T0[X Train Array]
XX3T1[Y Train Array] o--o X1T1[Y Train Array]
XX4T0[X Test Array] o--o X1T2[X Test Array]
XX4T1[Y Test Array] o--o X1T3[Y Test Array]
end

```

Processor: Default Supervised Learning



```mermaid
---
config:
    layout: elk
---
graph LR
subgraph GC0[Default Supervised Learning - Supervised Learning Block]
direction LR
subgraph GS0[Default Supervised Learning System]
subgraph G1[Fit Supervised Model - Default - Fit Supervised Model Block]
direction LR
X1[Fit Supervised Model - Default]
subgraph G1P[Ports]
direction TB
XX1P0[Model]
XX1P1[X Train Array]
XX1P2[Y Train Array]
end
XX1P0[Model] o--o X1
XX1P1[X Train Array] o--o X1
XX1P2[Y Train Array] o--o X1
subgraph G1T[Terminals]
direction TB
XX1T0[Model]
end
X1 o--o XX1T0[Model]
end
subgraph G2[Evaluate Supervised Model - Default - Evaluate Supervised Model Block]
direction LR
X2[Evaluate Supervised Model - Default]
subgraph G2P[Ports]
direction TB
XX2P0[Model]
XX2P1[X Test Array]
XX2P2[Y Test Array]
end
XX2P0[Model] o--o X2
XX2P1[X Test Array] o--o X2
XX2P2[Y Test Array] o--o X2
subgraph G2T[Terminals]
direction TB
XX2T0[Evaluation Metrics]
end
X2 o--o XX2T0[Evaluation Metrics]
end
subgraph G3[No Post-processing - Post-processing Block]
direction LR
X3[No Post-processing]
subgraph G3P[Ports]
direction TB
XX3P0[Evaluation Metrics]
end
XX3P0[Evaluation Metrics] o--o X3
subgraph G3T[Terminals]
direction TB
XX3T0[Evaluation Metrics]
end
X3 o--o XX3T0[Evaluation Metrics]
end
XX1T0[Model] ---> XX2P0[Model]
XX2T0[Evaluation Metrics] ---> XX3P0[Evaluation Metrics]
end
subgraph GC0P[Ports]
direction TB
X1P0[Model]
X1P1[X Train Array]
X1P2[Y Train Array]
X1P3[X Test Array]
X1P4[Y Test Array]
end
X1P0[Model] o--o XX1P0[Model]
X1P1[X Train Array] o--o XX1P1[X Train Array]
X1P2[Y Train Array] o--o XX1P2[Y Train Array]
X1P3[X Test Array] o--o XX2P1[X Test Array]
X1P4[Y Test Array] o--o XX2P2[Y Test Array]
subgraph GC0T[Terminals]
direction TB
X1T0[Evaluation Metrics]
end
XX3T0[Evaluation Metrics] o--o X1T0[Evaluation Metrics]
end

```